In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm
from data import get_dichalcogenides_innopolis_202105

In [3]:
structures = get_dichalcogenides_innopolis_202105()

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [4]:
import matplotlib

from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

import tensorflow as tf
import numpy as np

In [5]:
import wandb
from wandb.keras import WandbCallback

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [8]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [9]:
config = wandb.config
config.target = "energy_per_atom"

In [10]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc, npass=5)

In [11]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=True)
model.target_scaler = scaler

In [ ]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=1000, verbose=1, patience=1000, prev_model="callback/val_mae_00085_0.034367.hdf5")

Epoch 1/1000


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 79s 3s/step - loss: 0.0510


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 0.04380, saving model to callback/val_mae_00001_0.043804.hdf5


Epoch 2/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0451


...

Epoch 3/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0660
Epoch 4/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0889
Epoch 5/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0583
Epoch 6/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0486
Epoch 7/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0521


INFO:megnet.callbacks:
Epoch 00007: val_mae improved from 0.03522 to 0.03431, saving model to callback/val_mae_00007_0.034310.hdf5


Epoch 8/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0380
Epoch 9/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0391
Epoch 10/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0471
Epoch 11/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0348
Epoch 12/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0482
Epoch 13/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0548
Epoch 14/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0482


INFO:megnet.callbacks:
Epoch 00014: val_mae improved from 0.03431 to 0.03366, saving model to callback/val_mae_00014_0.033663.hdf5


Epoch 15/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0494
Epoch 16/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0372
Epoch 17/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0551
Epoch 18/1000
21/21 [==============================] - 56s 3s/step - loss: 0.0487
Epoch 19/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0489
Epoch 20/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0438
Epoch 21/1000
21/21 [==============================] - 55s 3s/step - loss: 0.0429
Epoch 22/1000
21/21 [==============================] - 57s 3s/step - loss: 0.0357
Epoch 23/1000
21/21 [==============================] - 57s 3s/step - loss: 0.0623
Epoch 24/1000
21/21 [==============================] - 58s 3s/step - loss: 0.0478
Epoch 25/1000
21/21 [==============================] - 57s 3s/step - loss: 0.0453
Epoch 26/1000
21/21 [==============================] - 57s 3s/step - loss: 0.0609
Epoch 27/1000
21

In [19]:
model.save_weights("megnet-05-data-E-per-atom-1k-epochs")